In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
from DataMatrix import generate_data_matrix
from scipy.stats import mode
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
)

import warnings

warnings.filterwarnings("ignore")

In [ ]:
X_train1, y_train1, X_test1, y_test1 = generate_data_matrix(method="mean")
X_train2, y_train2, X_test2, y_test2 = generate_data_matrix(method="flatten")

X1 = X_test1
y1 = y_test1
X2 = X_test2
y2 = y_test2

number_of_clusters = 0

print("X_test shape: ", X_test1.shape)
print("y_train shape: ", y_train1.shape)
print("X_test shape: ", X_test1.shape)
print("y_test shape: ", y_test1.shape)

### DBSCAN Algorithm


In [ ]:
def get_DBSCAN(X, min_samples, eps):
    # Initialize labels
    labels = np.zeros(X.shape[0])

    # C is the cluster counter
    C = 0
    for i in range(X.shape[0]):

        # Skip if already labeled
        if labels[i] != 0:
            continue

        # Find neighbors within eps
        neighbors = np.where(np.linalg.norm(X - X[i], axis=1) <= eps)[0]

        # Mark as noise
        if len(neighbors) < min_samples:
            labels[i] = -1
            continue

        # New cluster
        C += 1

        # Assign cluster label to point
        labels[i] = C

        # Set of points to expand
        S = list(neighbors)

        i = 0
        while i < len(S):
            j = S[i]
            if labels[j] == -1:
                labels[j] = C
            elif labels[j] == 0:
                labels[j] = C
                neighbors_j = np.where(np.linalg.norm(X - X[j], axis=1) <= eps)[0]
                if len(neighbors_j) >= min_samples:
                    S += list(set(neighbors_j) - set(S))
            i += 1
    # print("Number of Clusters: ", C)
    return labels

#### Labels Mapping Function with Ground Truth Labels with Majority Voting


In [ ]:
def map_labels(y_true, y_pred):
    # Get the unique labels in y_pred
    unique_labels = np.unique(y_pred)

    # Create a dictionary to map the labels
    label_map = {}

    for label in unique_labels:
        if label != -1:  # Skip noise points
            # Find the indices of points in this cluster
            indices = np.where(y_pred == label)[0]

            # Get the actual labels of these points
            actual_labels = y_true[indices]

            # Find the most common actual label
            majority_label = mode(actual_labels).mode

            # If majority_label is an array, take the first element
            if isinstance(majority_label, np.ndarray):
                majority_label = majority_label[0]

            # Map the predicted label to the majority label
            label_map[label] = majority_label

    # Map the labels in y_pred
    y_pred = np.array([label_map.get(label, -1) for label in y_pred])

    return y_pred

#### Entropy Calculation Function


In [ ]:
def entropy_score(y_true, y_pred):
    totalEntropy = 0
    y_pred = y_pred.astype(int)
    number_of_clusters = np.max(y_pred) + 1
    for i in range(number_of_clusters):
        entropy = 0
        # Getting the label indices of the points in the cluster
        labels = np.where(y_pred == i)

        # Getting the actual labels of the points in the cluster
        labels = y_true[labels]

        # Getting the counts of each label in each cluster
        labels, counts = np.unique(labels, return_counts=True)

        entropy = -np.sum(counts / np.sum(counts) * np.log2(counts / np.sum(counts)))

        totalEntropy += entropy

    # Dividing by the number of clusters to get the average conditional entropy
    totalEntropy /= number_of_clusters
    return totalEntropy

### Hyperparamters


In [ ]:
# Method 1 testset best hyperparameters
n_samples1 = 2
eps1 = 1.8

# Method 2 testset best hyperparameters
n_samples2 = 10
eps2 = 5

### Implemented DBSCAN


#### Method 1 (Mean)


In [ ]:
y_pred = get_DBSCAN(X1, n_samples1, eps1)
y_pred_mapped = map_labels(y1, y_pred)

# Compute the accuracy
precision = precision_score(y1, y_pred_mapped, average="weighted")
recall = recall_score(y1, y_pred_mapped, average="weighted")
f1 = f1_score(y1, y_pred_mapped, average="weighted")
entropy = entropy_score(y1, y_pred_mapped)

print("Implemented DBSCAN Evaluation Metrics:")
print("Precision: {:.3f}%".format(precision * 100))
print("Recall:    {:.3f}%".format(recall * 100))
print("F1-Score:  {:.3f}%".format(f1 * 100))
print("Entropy:   {:.3f}".format(entropy))

#### Method 2 (Flatten)


In [ ]:
y_pred = get_DBSCAN(X2, n_samples2, eps2)
y_pred_mapped = map_labels(y2, y_pred)

# Compute the accuracy
precision = precision_score(y2, y_pred_mapped, average="weighted")
recall = recall_score(y2, y_pred_mapped, average="weighted")
f1 = f1_score(y2, y_pred_mapped, average="weighted")
entropy = entropy_score(y2, y_pred_mapped)

print("Implemented DBSCAN Evaluation Metrics:")
print("Precision: {:.3f}%".format(precision * 100))
print("Recall:    {:.3f}%".format(recall * 100))
print("F1-Score:  {:.3f}%".format(f1 * 100))
print("Entropy:   {:.3f}".format(entropy))

### DBSCAN in Scikit-Learn


#### Method 1 (Mean)


In [ ]:
dbscan1 = DBSCAN(eps=eps1, min_samples=n_samples1)

# Fit the model to the data
dbscan1.fit(X1)

# Print the cluster labels for each data point
y_pred_sklearn = dbscan1.labels_
num_clusters = len(np.unique(y_pred_sklearn[y_pred_sklearn != -1]))
print("Number of clusters: ", num_clusters)
y_pred_sklearn_mapped = map_labels(y1, y_pred_sklearn)

precision = precision_score(y1, y_pred_sklearn_mapped, average="weighted")
recall = recall_score(y1, y_pred_sklearn_mapped, average="weighted")
f1 = f1_score(y1, y_pred_sklearn_mapped, average="weighted")
entropy = entropy_score(y1, y_pred_sklearn_mapped)

print("Sklearn DBSCAN Evaluation Metrics:")
print("Precision: {:.3f}%".format(precision * 100))
print("Recall:    {:.3f}%".format(recall * 100))
print("F1-Score:  {:.3f}%".format(f1 * 100))
print("Entropy:   {:.3f}".format(entropy))

#### Method 2 (Flatten)


In [ ]:
dbscan2 = DBSCAN(eps=eps2, min_samples=n_samples2)

# Fit the model to the data
dbscan2.fit(X2)

# Print the cluster labels for each data point
y_pred_sklearn = dbscan2.labels_
num_clusters = len(np.unique(y_pred_sklearn[y_pred_sklearn != -1]))
print("Number of clusters: ", num_clusters)
y_pred_sklearn_mapped = map_labels(y2, y_pred_sklearn)

precision = precision_score(y2, y_pred_sklearn_mapped, average="weighted")
recall = recall_score(y2, y_pred_sklearn_mapped, average="weighted")
f1 = f1_score(y2, y_pred_sklearn_mapped, average="weighted")
entropy = entropy_score(y2, y_pred_sklearn_mapped)

print("Sklearn DBSCAN Evaluation Metrics:")
print("Precision: {:.3f}%".format(precision * 100))
print("Recall:    {:.3f}%".format(recall * 100))
print("F1-Score:  {:.3f}%".format(f1 * 100))
print("Entropy:   {:.3f}".format(entropy))